In [1]:
import tensorflow as tf
from utils.data import *
from utils.lstm_rnn import *
import os
import pandas as pd
if not os.path.isdir("save"):
    os.mkdir("save")
##### config #####
save_dataset = False # 儲存dataset，true: 生成dataset、false: 使用save/ 儲存的
window_size = 60
shift = 10
repeat = 100



##### config #####
dataPoint_per_signal = (180-window_size+1)//shift  * window_size
print("datapoint per signal:", dataPoint_per_signal)

datapoint per signal: 720


In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
for i in range(len(physical_devices)):
    tf.config.experimental.set_memory_growth(physical_devices[i], enable=True)

if save_dataset:
    X_raw, y = generate_time_series_dataset(window_size=window_size, shift=shift, repeat=repeat)
    #X = tf.reshape(X_raw, [y.shape[0],timestep,dataPoint_per_signal//timestep])
    np.save("save/X_raw-time.npy", X_raw.numpy())
    #np.save("save/X-time.npy", X.numpy())
    np.save("save/y-time.npy", y)
else:
    X_raw = np.load("save/X_raw-time.npy")
    #X = np.load("save/X-time.npy")
    y = np.load("save/y-time.npy")

X_raw = tf.convert_to_tensor(X_raw)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [3]:
"""
timestep=3

X = tf.reshape(X_raw, [y.shape[0],timestep,dataPoint_per_signal//timestep])
print(X.shape)
train_dataset, val_dataset, test_dataset = split_dataset(X, y)
X.shape[2]
"""

(160000, 3, 240)
112000 24000 24000
train: 112000
val: 24000
test: 24000
new train 112000


240

In [5]:
try_timesteps=[3,6]#list(range(3,18,3))
try_units=[100]#[50,100,200,300,180]#list(range(10,200,20))
try_epochs=[100]

"""
try_timesteps=[3]
try_units=[180]#list(range(10,200,20))
try_epochs=[150]
"""
df_all = pd.DataFrame([], columns=[["loss", "out_1_loss", "out_2_loss", "out_1_acc", "out_2_acc", "timesteps", "units", "epoch"]])
for timesteps in try_timesteps:
    for units in try_units:
        for epoch in try_epochs:
            X = tf.reshape(X_raw, [y.shape[0],timesteps,dataPoint_per_signal//timesteps])
            train_dataset, val_dataset, test_dataset = split_dataset(X, y)
            
            model = construct_model(timesteps=timesteps, data_dim = X.shape[2], units=units)
            model, history = compile_fit_model(model,epochs=epoch,train_dataset=train_dataset,val_dataset=val_dataset)
            model.save("saved_model/lstm-rnn-unit("+str(units)+")-timesteps("+str(timesteps)+")-epoch("+str(epoch)+")")
            results = model.evaluate(test_dataset)
            results.extend([timesteps, units, epoch])
            df = pd.DataFrame([results], columns=[["loss", "out_1_loss", "out_2_loss", "out_1_acc", "out_2_acc", "timesteps", "units", "epoch"]])
            df_all = df_all.append(df)
df_all.to_csv("save/result.csv")

112000 24000 24000
train: 112000
val: 24000
test: 24000
new train 112000
Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 3, 240)]     0                                            
__________________________________________________________________________________________________
vfc_1_1 (LSTM)                  [(None, 3, 100), (No 136400      input_5[0][0]                    
__________________________________________________________________________________________________
vfc_1_2 (LSTM)                  [(None, 100), (None, 80400       vfc_1_1[0][0]                    
__________________________________________________________________________________________________
vfc_2_1 (LSTM)                  (None, 3, 100)       136400      input_5[0][0]                    
                   

28/28 [==============================] - 1s 43ms/step - loss: 0.1564 - out_1_loss: 0.0933 - out_2_loss: 0.0631 - out_1_acc: 0.9494 - out_2_acc: 0.9664 - val_loss: 0.2236 - val_out_1_loss: 0.1160 - val_out_2_loss: 0.1077 - val_out_1_acc: 0.9371 - val_out_2_acc: 0.9524
Epoch 20/100
28/28 [==============================] - 1s 43ms/step - loss: 0.1530 - out_1_loss: 0.0919 - out_2_loss: 0.0611 - out_1_acc: 0.9500 - out_2_acc: 0.9670 - val_loss: 0.2262 - val_out_1_loss: 0.1161 - val_out_2_loss: 0.1101 - val_out_1_acc: 0.9370 - val_out_2_acc: 0.9521
Epoch 21/100
28/28 [==============================] - 1s 43ms/step - loss: 0.1497 - out_1_loss: 0.0905 - out_2_loss: 0.0593 - out_1_acc: 0.9508 - out_2_acc: 0.9680 - val_loss: 0.2285 - val_out_1_loss: 0.1165 - val_out_2_loss: 0.1120 - val_out_1_acc: 0.9367 - val_out_2_acc: 0.9520
Epoch 22/100
28/28 [==============================] - 1s 43ms/step - loss: 0.1465 - out_1_loss: 0.0892 - out_2_loss: 0.0573 - out_1_acc: 0.9515 - out_2_acc: 0.9689 - val_

28/28 [==============================] - 1s 45ms/step - loss: 0.1010 - out_1_loss: 0.0618 - out_2_loss: 0.0392 - out_1_acc: 0.9682 - out_2_acc: 0.9798 - val_loss: 0.3108 - val_out_1_loss: 0.1497 - val_out_2_loss: 0.1611 - val_out_1_acc: 0.9388 - val_out_2_acc: 0.9516
Epoch 49/100
28/28 [==============================] - 1s 45ms/step - loss: 0.0999 - out_1_loss: 0.0610 - out_2_loss: 0.0389 - out_1_acc: 0.9687 - out_2_acc: 0.9797 - val_loss: 0.3142 - val_out_1_loss: 0.1516 - val_out_2_loss: 0.1626 - val_out_1_acc: 0.9395 - val_out_2_acc: 0.9526
Epoch 50/100
28/28 [==============================] - 1s 43ms/step - loss: 0.0986 - out_1_loss: 0.0604 - out_2_loss: 0.0382 - out_1_acc: 0.9689 - out_2_acc: 0.9802 - val_loss: 0.3154 - val_out_1_loss: 0.1527 - val_out_2_loss: 0.1627 - val_out_1_acc: 0.9403 - val_out_2_acc: 0.9534
Epoch 51/100
28/28 [==============================] - 1s 44ms/step - loss: 0.0967 - out_1_loss: 0.0593 - out_2_loss: 0.0374 - out_1_acc: 0.9693 - out_2_acc: 0.9810 - val_

28/28 [==============================] - 1s 44ms/step - loss: 0.0852 - out_1_loss: 0.0523 - out_2_loss: 0.0329 - out_1_acc: 0.9742 - out_2_acc: 0.9837 - val_loss: 0.3785 - val_out_1_loss: 0.1873 - val_out_2_loss: 0.1913 - val_out_1_acc: 0.9408 - val_out_2_acc: 0.9519
Epoch 78/100
28/28 [==============================] - 1s 46ms/step - loss: 0.0837 - out_1_loss: 0.0516 - out_2_loss: 0.0321 - out_1_acc: 0.9745 - out_2_acc: 0.9843 - val_loss: 0.3845 - val_out_1_loss: 0.1893 - val_out_2_loss: 0.1952 - val_out_1_acc: 0.9400 - val_out_2_acc: 0.9524
Epoch 79/100
28/28 [==============================] - 1s 46ms/step - loss: 0.0831 - out_1_loss: 0.0518 - out_2_loss: 0.0313 - out_1_acc: 0.9739 - out_2_acc: 0.9844 - val_loss: 0.3889 - val_out_1_loss: 0.1920 - val_out_2_loss: 0.1969 - val_out_1_acc: 0.9403 - val_out_2_acc: 0.9515
Epoch 80/100
28/28 [==============================] - 1s 43ms/step - loss: 0.0820 - out_1_loss: 0.0511 - out_2_loss: 0.0308 - out_1_acc: 0.9743 - out_2_acc: 0.9849 - val_

Epoch 1/100
28/28 [==============================] - 2s 87ms/step - loss: 6.3468 - out_1_loss: 3.2074 - out_2_loss: 3.1394 - out_1_acc: 0.3381 - out_2_acc: 0.3356 - val_loss: 5.0850 - val_out_1_loss: 2.6215 - val_out_2_loss: 2.4636 - val_out_1_acc: 0.4380 - val_out_2_acc: 0.4133
Epoch 2/100
28/28 [==============================] - 1s 50ms/step - loss: 3.5909 - out_1_loss: 1.9684 - out_2_loss: 1.6225 - out_1_acc: 0.5525 - out_2_acc: 0.6322 - val_loss: 1.8886 - val_out_1_loss: 1.1585 - val_out_2_loss: 0.7300 - val_out_1_acc: 0.7323 - val_out_2_acc: 0.8540
Epoch 3/100
28/28 [==============================] - 2s 54ms/step - loss: 1.1509 - out_1_loss: 0.7369 - out_2_loss: 0.4139 - out_1_acc: 0.8386 - out_2_acc: 0.9199 - val_loss: 0.6962 - val_out_1_loss: 0.4544 - val_out_2_loss: 0.2419 - val_out_1_acc: 0.9090 - val_out_2_acc: 0.9455
Epoch 4/100
28/28 [==============================] - 1s 49ms/step - loss: 0.5243 - out_1_loss: 0.3479 - out_2_loss: 0.1764 - out_1_acc: 0.9231 - out_2_acc: 0.95

28/28 [==============================] - 1s 49ms/step - loss: 0.1530 - out_1_loss: 0.0861 - out_2_loss: 0.0668 - out_1_acc: 0.9508 - out_2_acc: 0.9626 - val_loss: 0.2154 - val_out_1_loss: 0.1162 - val_out_2_loss: 0.0992 - val_out_1_acc: 0.9388 - val_out_2_acc: 0.9533
Epoch 30/100
28/28 [==============================] - 2s 55ms/step - loss: 0.1511 - out_1_loss: 0.0854 - out_2_loss: 0.0657 - out_1_acc: 0.9515 - out_2_acc: 0.9631 - val_loss: 0.2171 - val_out_1_loss: 0.1166 - val_out_2_loss: 0.1005 - val_out_1_acc: 0.9386 - val_out_2_acc: 0.9534
Epoch 31/100
28/28 [==============================] - 1s 49ms/step - loss: 0.1488 - out_1_loss: 0.0845 - out_2_loss: 0.0644 - out_1_acc: 0.9513 - out_2_acc: 0.9641 - val_loss: 0.2195 - val_out_1_loss: 0.1169 - val_out_2_loss: 0.1026 - val_out_1_acc: 0.9384 - val_out_2_acc: 0.9523
Epoch 32/100
28/28 [==============================] - 1s 49ms/step - loss: 0.1474 - out_1_loss: 0.0840 - out_2_loss: 0.0634 - out_1_acc: 0.9517 - out_2_acc: 0.9646 - val_

28/28 [==============================] - 1s 48ms/step - loss: 0.1136 - out_1_loss: 0.0668 - out_2_loss: 0.0468 - out_1_acc: 0.9621 - out_2_acc: 0.9735 - val_loss: 0.2841 - val_out_1_loss: 0.1428 - val_out_2_loss: 0.1414 - val_out_1_acc: 0.9409 - val_out_2_acc: 0.9522
Epoch 59/100
28/28 [==============================] - 1s 49ms/step - loss: 0.1124 - out_1_loss: 0.0663 - out_2_loss: 0.0461 - out_1_acc: 0.9627 - out_2_acc: 0.9741 - val_loss: 0.2909 - val_out_1_loss: 0.1443 - val_out_2_loss: 0.1466 - val_out_1_acc: 0.9403 - val_out_2_acc: 0.9534
Epoch 60/100
28/28 [==============================] - 1s 49ms/step - loss: 0.1114 - out_1_loss: 0.0659 - out_2_loss: 0.0455 - out_1_acc: 0.9627 - out_2_acc: 0.9743 - val_loss: 0.2888 - val_out_1_loss: 0.1439 - val_out_2_loss: 0.1449 - val_out_1_acc: 0.9405 - val_out_2_acc: 0.9515
Epoch 61/100
28/28 [==============================] - 1s 50ms/step - loss: 0.1108 - out_1_loss: 0.0654 - out_2_loss: 0.0453 - out_1_acc: 0.9626 - out_2_acc: 0.9743 - val_

28/28 [==============================] - 1s 51ms/step - loss: 0.0951 - out_1_loss: 0.0569 - out_2_loss: 0.0383 - out_1_acc: 0.9691 - out_2_acc: 0.9791 - val_loss: 0.3453 - val_out_1_loss: 0.1711 - val_out_2_loss: 0.1742 - val_out_1_acc: 0.9408 - val_out_2_acc: 0.9525
Epoch 88/100
28/28 [==============================] - 1s 52ms/step - loss: 0.0953 - out_1_loss: 0.0568 - out_2_loss: 0.0385 - out_1_acc: 0.9690 - out_2_acc: 0.9793 - val_loss: 0.3560 - val_out_1_loss: 0.1765 - val_out_2_loss: 0.1795 - val_out_1_acc: 0.9411 - val_out_2_acc: 0.9525
Epoch 89/100
28/28 [==============================] - 2s 55ms/step - loss: 0.0941 - out_1_loss: 0.0563 - out_2_loss: 0.0378 - out_1_acc: 0.9691 - out_2_acc: 0.9797 - val_loss: 0.3540 - val_out_1_loss: 0.1758 - val_out_2_loss: 0.1782 - val_out_1_acc: 0.9405 - val_out_2_acc: 0.9522
Epoch 90/100
28/28 [==============================] - 1s 53ms/step - loss: 0.0930 - out_1_loss: 0.0557 - out_2_loss: 0.0373 - out_1_acc: 0.9699 - out_2_acc: 0.9798 - val_

In [ ]:
df_all

(180,)


(720,)

In [107]:

X_raw, y = generate_time_series_dataset(window_size=window_size, shift=shift, repeat=repeat)
print( X_raw.shape, y.shape )

num of freq combination (160000, 2)


KeyboardInterrupt: 

In [99]:

X.shape

TensorShape([160000, 3, 240])

In [81]:
window_size

60

#test function

In [177]:
# SHIFT的方法 using tensorflow
range_ds = tf.data.Dataset.range(6)
window_size = 3
shift = 1
windows = range_ds.window(window_size, shift=shift)
def sub_to_batch(sub):
    return sub.batch(window_size, drop_remainder=True)

i=0
for example in windows.flat_map(sub_to_batch):
    print(example.numpy())
    i+=1
print("shape", i)

[0 1 2]
[1 2 3]
[2 3 4]
[3 4 5]
shape 4


In [34]:
data = np.arange(180)
def sliding_window(data=None, window_size = 6, shift = 1):
    return np.array(list( list(data[(x*shift):(x*shift)+window_size]) for x in range((data.shape[0]-window_size+1)//shift) ))

a = sliding_window(data, window_size = 60, shift = 10)
a

array([[  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59],
       [ 10,  11,  12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,
         23,  24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,
         36,  37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,
         49,  50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,
         62,  63,  64,  65,  66,  67,  68,  69],
       [ 20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,
         33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,
         46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,
         59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,
         72,  73,  74,  75

In [37]:
a.flatten().shape

(720,)

In [45]:
a.reshape(-1)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  10,  11,  12,  13,  14,
        15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,
        30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  42,
        43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
        56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,
        69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  3

In [71]:
s = np_generate_signal([0,0],30)
print(s.shape)
s_window = sliding_window(data=s, window_size = 60, shift = 10).flatten()
s_window.shape

12